In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para el análisis descriptivo inicial de contraste de asociación importo el test Chi2 y el anova 
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

# Cargaríamos la librería scortecardpy si fuese estable (que no lo es)
#import scorecardpy as sc

# Librería para hacer la tramificación, agrupación y transformación WOE
from optbinning import Scorecard, BinningProcess, OptimalBinning
from optbinning.scorecard import plot_auc_roc, plot_cap, plot_ks


# Scikit-learn para dividir la muestra y para estimar el modelo de regresión logística (sólo si no se quiere utilizar
# la función optbinning.scorecard que ya lo incropora)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

(CVXPY) Mar 27 12:15:57 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.9.3963). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 27 12:15:57 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.9.3963). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
# Cargamos los datos 
dt=pd.read_excel('DatosPractica_Scoring.xlsx')

In [3]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        1319 non-null   int64  
 1   Cardhldr  1285 non-null   float64
 2   default   994 non-null    float64
 3   Age       1319 non-null   float64
 4   Income    1319 non-null   float64
 5   Exp_Inc   1319 non-null   float64
 6   Avgexp    1319 non-null   float64
 7   Ownrent   1319 non-null   int64  
 8   Selfempl  1319 non-null   int64  
 9   Depndt    1319 non-null   int64  
 10  Inc_per   1319 non-null   float64
 11  Cur_add   1319 non-null   int64  
 12  Major     1319 non-null   int64  
 13  Active    1319 non-null   int64  
dtypes: float64(7), int64(7)
memory usage: 144.4 KB


In [4]:
dt.drop(labels='ID',inplace=True, axis=1)

dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Cardhldr  1285 non-null   float64
 1   default   994 non-null    float64
 2   Age       1319 non-null   float64
 3   Income    1319 non-null   float64
 4   Exp_Inc   1319 non-null   float64
 5   Avgexp    1319 non-null   float64
 6   Ownrent   1319 non-null   int64  
 7   Selfempl  1319 non-null   int64  
 8   Depndt    1319 non-null   int64  
 9   Inc_per   1319 non-null   float64
 10  Cur_add   1319 non-null   int64  
 11  Major     1319 non-null   int64  
 12  Active    1319 non-null   int64  
dtypes: float64(7), int64(6)
memory usage: 134.1 KB


In [110]:
dt_acept = dt.copy()
dt_acept = dt_acept[dt_acept['Cardhldr'] == 1]



In [111]:
variable="Active"
X=dt_acept[variable].values
Y=dt_acept['default'].values

optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

optb.fit(X, Y)
optb.splits

array([ 2.5,  5.5,  6.5,  9.5, 11.5, 18.5])

In [112]:
binning_table = optb.binning_table
binning_table.build()

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 2.50)",259,0.260563,256,3,0.011583,2.299735,0.595158,0.061393
1,"[2.50, 5.50)",203,0.204225,184,19,0.093596,0.123666,0.002974,0.000372
2,"[5.50, 6.50)",58,0.058350,52,6,0.103448,0.012654,0.000009,0.000001
3,"[6.50, 9.50)",168,0.169014,149,19,0.113095,-0.087323,0.001334,0.000167
4,"[9.50, 11.50)",83,0.083501,72,11,0.132530,-0.26806,0.006667,0.000831
5,"[11.50, 18.50)",168,0.169014,135,33,0.196429,-0.738063,0.122240,0.014942
6,"[18.50, inf)",55,0.055332,42,13,0.236364,-0.97411,0.075795,0.009117
7,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
8,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,994,1.000000,890,104,0.104628,,0.804177,0.086823


In [113]:
print("IV= ", binning_table.iv.round(3))
print("Gini= ", binning_table.gini.round(3))

IV=  0.804
Gini=  0.406
